# LSTM을 사용한 모델링

In [3]:
import pandas as pd
import numpy as np
from string import punctuation # 구두점

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [4]:
df = pd.read_csv("../input/ArticlesApril2018.csv")

df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1324 entries, 0 to 1323
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   articleID         1324 non-null   object
 1   articleWordCount  1324 non-null   int64 
 2   byline            1324 non-null   object
 3   documentType      1324 non-null   object
 4   headline          1324 non-null   object
 5   keywords          1324 non-null   object
 6   multimedia        1324 non-null   int64 
 7   newDesk           1324 non-null   object
 8   printPage         1324 non-null   int64 
 9   pubDate           1324 non-null   object
 10  sectionName       1324 non-null   object
 11  snippet           1324 non-null   object
 12  source            1324 non-null   object
 13  typeOfMaterial    1324 non-null   object
 14  webURL            1324 non-null   object
dtypes: int64(3), object(12)
memory usage: 155.3+ KB


In [7]:
# headline 결측치 확인
df["headline"].isnull().any()

False

In [11]:
df["headline"].head() # Unknown 값 제거 필요

0    Former N.F.L. Cheerleaders’ Settlement Offer: ...
1    E.P.A. to Unveil a New Rule. Its Effect: Less ...
2                              The New Noma, Explained
3                                              Unknown
4                                              Unknown
Name: headline, dtype: object

In [13]:
mask = df["headline"] == "Unknown"
tar = df[mask]

len(tar)

110

In [16]:
idx = tar.index
headline = df["headline"].drop(idx).values

headline # Unknown 제거

array(['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
       'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
       'The New Noma, Explained', ..., 'Anna, Llama and Me',
       'Gen. Michael Hayden Has One Regret: Russia',
       'There Is Nothin’ Like a Tune'], dtype=object)

In [20]:
# Preprocessing
def preprocessing(raw_sent):
    """구두점 제거 및 단어의 소문자화"""
    preprocessed_sent = raw_sent.encode("utf8").decode("ascii", "ignore")
    res = "".join(word for word in preprocessed_sent if word not in punctuation).lower()
    return res

preprocessed_headline = [preprocessing(item) for item in headline]

preprocessed_headline[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [21]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_headline)
vocab_size = len(tokenizer.word_index) + 1

vocab_size

3494

In [24]:
sequences = []

for sent in preprocessed_headline:
    encoded = tokenizer.texts_to_sequences([sent])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i + 1]
        sequences.append(sequence)

sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [31]:
# Padding and Labeling
idx_to_word = {}
for k, v in tokenizer.word_index.items():
    idx_to_word[k] = v

max_len = max(len(sequence) for sequence in sequences) 
sequences = pad_sequences(sequences, maxlen=max_len, padding="pre")

X_train = sequences[:, :-1]
y_train = sequences[:, -1]
y_train = to_categorical(y_train, num_classes=vocab_size)

X_train[:3], y_train[:3]

(array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,  99],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,  99, 269],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,  99, 269, 371]], dtype=int32),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32))

In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [34]:
# Modeling
embedding_dim = 10
hidden_size = 128

model = Sequential()
model.add((Embedding(vocab_size, embedding_dim)))
model.add(LSTM(hidden_size))
model.add(Dense(vocab_size, activation="softmax")) # 다중 분류
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=100, verbose=2)

Epoch 1/200


2023-07-17 19:23:37.593684: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-07-17 19:23:38.414606: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-07-17 19:23:38.603022: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-07-17 19:23:39.790920: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


244/244 - 9s - loss: 7.6453 - accuracy: 0.0292 - 9s/epoch - 37ms/step
Epoch 2/200
244/244 - 5s - loss: 7.1045 - accuracy: 0.0304 - 5s/epoch - 21ms/step
Epoch 3/200
244/244 - 5s - loss: 6.9688 - accuracy: 0.0329 - 5s/epoch - 21ms/step
Epoch 4/200
244/244 - 5s - loss: 6.8452 - accuracy: 0.0418 - 5s/epoch - 22ms/step
Epoch 5/200
244/244 - 6s - loss: 6.6919 - accuracy: 0.0425 - 6s/epoch - 24ms/step
Epoch 6/200
244/244 - 6s - loss: 6.5160 - accuracy: 0.0477 - 6s/epoch - 24ms/step
Epoch 7/200
244/244 - 6s - loss: 6.3300 - accuracy: 0.0510 - 6s/epoch - 24ms/step
Epoch 8/200
244/244 - 6s - loss: 6.1271 - accuracy: 0.0578 - 6s/epoch - 25ms/step
Epoch 9/200
244/244 - 6s - loss: 5.9266 - accuracy: 0.0628 - 6s/epoch - 25ms/step
Epoch 10/200
244/244 - 6s - loss: 5.7360 - accuracy: 0.0693 - 6s/epoch - 26ms/step
Epoch 11/200
244/244 - 6s - loss: 5.5571 - accuracy: 0.0711 - 6s/epoch - 25ms/step
Epoch 12/200
244/244 - 6s - loss: 5.3889 - accuracy: 0.0757 - 6s/epoch - 23ms/step
Epoch 13/200
244/244 - 6s

In [35]:
def sent_generator(model, tokenizer, current_word, n):
    init_word = current_word
    sentence = ""

    for _ in range(n):
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len, padding="pre")

        res = model.predict(encoded, verbose=0)
        res = np.argmax(res, axis=1)

        for word, idx in tokenizer.word_index.items():
            if idx == res:
                break
        
        current_word = current_word + " " + word
        sentence = sentence + " " + word

    sentence = init_word + sentence
    return sentence

print(sent_generator(model, tokenizer, "i", 10)) # i로 시작하면서 길이가 10인 문장 생성
print(sent_generator(model, tokenizer, "how", 10)) # how로 시작하면서 길이가 10인 문장 생성

2023-07-17 19:52:33.729309: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-07-17 19:52:33.809714: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


i cant jump ship from facebook yet takes of an dept
how do you feel about being told to smile focus of
